In [1]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np

# Configure OSM tags
ox.settings.useful_tags_node = ['highway', 'traffic_signals', 'stop', 'crossing', 'give_way']
ox.settings.useful_tags_way = [
    'highway', 'name', 'ref', 'lanes', 'maxspeed', 'oneway', 
    'bridge', 'tunnel', 'layer', 'junction',
    'turn:lanes', 'surface', 'lit', 'width'
]

# Download network
print("Downloading Palm Beach County road network...") 
G = ox.graph_from_place("Palm Beach County, Florida, USA", network_type="drive_service", simplify=False) #simplify = false because we need the highest quantity of attributes, stop signs, yield, etc...
G = ox.project_graph(G, to_crs="EPSG:26917") 
nodes, edges = ox.graph_to_gdfs(G)
nodes = nodes.reset_index() 
edges = edges.reset_index()

print("Downloading landuse polygons...")
landuse_tags = {"landuse": ["residential", "retail", "commercial", "industrial", 
                            "construction", "education", "institutional", "forest"]}
landuse_polys = ox.features_from_place("Palm Beach County, Florida, USA", tags=landuse_tags)
landuse_polys = landuse_polys.to_crs("EPSG:26917")
landuse_polys = landuse_polys.reset_index()

# Flatten nested columns OSM sometimes returns lists
def stringify_and_flatten(val):
    if isinstance(val, (list, np.ndarray)):
        return "|".join(map(str, val))
    if isinstance(val, (int, float, np.integer, np.floating)):
        return val
    if pd.isna(val):
        return np.nan
    return str(val)

skip = {'geometry', 'u', 'v', 'key', 'osmid'}

def clean_osm_columns(df):
    for col in df.columns:
        if col in skip: 
            continue
        df[col] = df[col].apply(stringify_and_flatten)
        df[col] = df[col].astype(str).replace('nan', np.nan)
    return df

print("Cleaning datasets...")
nodes_clean = clean_osm_columns(nodes.copy())
edges_clean = clean_osm_columns(edges.copy())
landuse_polys = clean_osm_columns(landuse_polys)

# Join traffic controls to road segments
# OSM stores these as node attributes, but we need them on edges for analysis
print("Joining traffic controls to edges...")
nodes['has_stop'] = (nodes['highway'] == 'stop').astype(int)
nodes['has_signal'] = (nodes['highway'] == 'traffic_signals').astype(int)
nodes['has_crossing'] = nodes['crossing'].notna().astype(int)
nodes['has_give_way'] = (nodes['highway'] == 'give_way').astype(int)

node_lookup = nodes.set_index('osmid')[['has_stop', 'has_signal', 'has_crossing', 'has_give_way']]

# Map controls to both ends of each edge (u = start, v = end)
edges_clean['OSM_has_stop_u'] = edges['u'].map(node_lookup['has_stop']).fillna(0).astype(int)
edges_clean['OSM_has_signal_u'] = edges['u'].map(node_lookup['has_signal']).fillna(0).astype(int)
edges_clean['OSM_has_crossing_u'] = edges['u'].map(node_lookup['has_crossing']).fillna(0).astype(int)
edges_clean['OSM_has_give_way_u'] = edges['u'].map(node_lookup['has_give_way']).fillna(0).astype(int)

edges_clean['OSM_has_stop_v'] = edges['v'].map(node_lookup['has_stop']).fillna(0).astype(int)
edges_clean['OSM_has_signal_v'] = edges['v'].map(node_lookup['has_signal']).fillna(0).astype(int)
edges_clean['OSM_has_crossing_v'] = edges['v'].map(node_lookup['has_crossing']).fillna(0).astype(int)
edges_clean['OSM_has_give_way_v'] = edges['v'].map(node_lookup['has_give_way']).fillna(0).astype(int)

edges_clean['has_stop_at_ends'] = (edges_clean['OSM_has_stop_u'] | edges_clean['OSM_has_stop_v']).astype(int)
edges_clean['has_signal_at_ends'] = (edges_clean['OSM_has_signal_u'] | edges_clean['OSM_has_signal_v']).astype(int)
edges_clean['has_crossing_at_ends'] = (edges_clean['OSM_has_crossing_u'] | edges_clean['OSM_has_crossing_v']).astype(int)
edges_clean['has_give_way_at_ends'] = (edges_clean['OSM_has_give_way_u'] | edges_clean['OSM_has_give_way_v']).astype(int)

print(f"\nExtracted {len(nodes):,} nodes and {len(edges):,} edges")
print(f"Traffic controls: {nodes['has_stop'].sum():,} stops, {nodes['has_signal'].sum():,} signals")
print(f"Landuse polygons: {len(landuse_polys):,}")

# Save to parquet
nodes_clean.to_parquet("bronze_osm_nodes.parquet")
edges_clean.to_parquet("bronze_osm_network.parquet")
landuse_polys.to_parquet("bronze_osm_landuse.parquet")
print("\nBronze layer saved")

Cleaning datasets...
Joining traffic controls to edges...

Extracted 493,737 nodes and 947,921 edges
Traffic controls: 4,696 stops, 2,863 signals
Landuse polygons: 1,829

Bronze layer saved


In [4]:
import os
gdb_path = "DOTShapesFGDB.gdb"
layers = gpd.list_layers(gdb_path)

os.makedirs("bronze_fdot_layers", exist_ok=True)
fdot_metadata = []

for layer_name in layers['name']:
    try:
        # County 93 is Palm Beach
        query = "COUNTY = 'PALM BEACH' OR COUNTYDOT = '93'"
        
        df = gpd.read_file(
            gdb_path, 
            layer=layer_name, 
            where=query, 
            engine='pyogrio'
        )
        
        if not df.empty:
            # we need it this way for the spatial join later on, it's not a big deal and can be converted to degrees in the silver layer. 
            df = df.to_crs(epsg=26917)
            
            file_path = f"bronze_fdot_layers/{layer_name}.parquet"
            df.to_parquet(file_path)

            # can sometimes miss the "milepost" not a big deal, we don't really need it. (x coordinate, y coordinate, m milepost metadata unecessary for the spatial join, it's for potholes and shit)
            fdot_metadata.append({'layer': layer_name, 'count': len(df), 'path': file_path})
            print(f"Layer Saved: {layer_name} ({len(df)} rows)")

    except Exception as e:
        print(f"Error extracting {layer_name}: {e}")


# to note, some of columns are missing, (basemap_routes, basemap_arcs). We don't need them because we are using OSM as our skeleton, making a new one from fdot would probably be hell.
pd.DataFrame(fdot_metadata).to_parquet("bronze_fdot_manifest.parquet", index=False)
print("\nBronze Extraction Complete.")

Layer Saved: aadt (1334 rows)
Layer Saved: access_management (115 rows)
Error extracting basemap_arcs: Invalid SQL query for layer 'b'basemap_arcs'': 'COUNTY = 'PALM BEACH' OR COUNTYDOT = '93''
Layer Saved: basemap_route_road (1197 rows)
Error extracting basemap_routes: Invalid SQL query for layer 'b'basemap_routes'': 'COUNTY = 'PALM BEACH' OR COUNTYDOT = '93''


/home/wizard/anaconda3/envs/ox/lib/python3.14/site-packages/pyogrio/raw.py:200: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured MultiLineString' is converted to 'MultiLineString'
  return ogr_read(
/home/wizard/anaconda3/envs/ox/lib/python3.14/site-packages/pyogrio/raw.py:200: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured MultiLineString' is converted to 'MultiLineString'
  return ogr_read(


Layer Saved: bike_lane (2042 rows)
Layer Saved: bike_slot (1644 rows)
Layer Saved: box_culverts (18 rows)
Layer Saved: bridges (542 rows)
Layer Saved: county_roads (21 rows)
Layer Saved: divided (589 rows)
Layer Saved: faccross (371 rows)
Layer Saved: fahwysys (1086 rows)
Layer Saved: funclass (1142 rows)
Layer Saved: hpms (118 rows)
Layer Saved: inside_shoulder_type (3491 rows)
Layer Saved: inside_shoulder_width (3017 rows)
Layer Saved: interchange (45 rows)
Layer Saved: intersection (16250 rows)
Layer Saved: interstates (1 rows)
Layer Saved: localnam (1334 rows)
Layer Saved: maintenance_boundary (260 rows)
Layer Saved: maxspeed (349 rows)
Layer Saved: median_type (4487 rows)
Layer Saved: median_width (4496 rows)
Layer Saved: mpoarea (1250 rows)
Layer Saved: national_highway_freight_network (10 rows)
Layer Saved: national_network (3 rows)
Layer Saved: nhs (61 rows)
Layer Saved: number_of_lanes (5461 rows)
Layer Saved: off_system (592 rows)
Layer Saved: on_system (57 rows)
Layer Saved: